# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix (q=0,ω=0) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.233096356919                   -0.50    6.0
  2   -7.250003805877       -1.77       -1.40    1.0
  3   -7.251230766897       -2.91       -1.92    4.0
  4   -7.251145176002   +   -4.07       -2.01    4.0
  5   -7.251295350161       -3.82       -2.35    3.0
  6   -7.251337022151       -4.38       -2.97    2.0
  7   -7.251338758578       -5.76       -3.93    2.0
  8   -7.251338782460       -7.62       -3.95    5.0
  9   -7.251338797524       -7.82       -4.48    3.0
 10   -7.251338798391       -9.06       -4.82    2.0
 11   -7.251338798695       -9.52       -5.53    3.0
 12   -7.251338798704      -11.09       -5.87    2.0
 13   -7.251338798704      -12.37       -6.07    1.0
 14   -7.251338798704      -12.37       -6.44    3.0
 15   -7.251338798705      -13.10       -6.89    2.0
 16   -7.251338798705      -13.97       -7.19    2.0
 17   -7.251338798705      -14.57       -8.02 

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Apply ε† = 1 - χ0 (vc + fxc)

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end

eps_fun (generic function with 1 method)

eager diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.049770845578473895
[ Info: Arnoldi iteration step 2: normres = 0.578182068445808
[ Info: Arnoldi iteration step 3: normres = 0.8782963057467941
[ Info: Arnoldi iteration step 4: normres = 0.24887614991288218
[ Info: Arnoldi iteration step 5: normres = 0.577844527719869
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (3.27e-02, 8.09e-02, 5.30e-01, 2.13e-01, 1.11e-02)
[ Info: Arnoldi iteration step 6: normres = 0.22990754962794702
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (6.36e-03, 1.37e-01, 1.33e-01, 9.45e-02, 3.28e-02)
[ Info: Arnoldi iteration step 7: normres = 0.10569735497873059
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (3.28e-04, 1.36e-02, 1.15e-02, 7.34e-02, 6.84e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10982453541751029
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.58e-